&nbsp;
&nbsp;
![](../_resources/images/e2eai-5.jpg)

# MCP and Model / Agent Communication

## 🌐 What is MCP?

**Model Context Protocol (MCP)** is an open-source standard for connecting AI applications to external systems. Think of it like **USB-C for AI** - a standardized way to connect.

### 🔌 Key Concepts:

**MCP Server** - Provides resources and tools to clients
**MCP Client** - AI application that uses those tools
**Protocol** - Standardized JSON-RPC 2.0 communication

### 📊 MCP Ecosystem:

There are several agent communication protocols emerging:
- **[MCP](https://modelcontextprotocol.io)** (Anthropic) - Most widely adopted
- **[Agent2Agent](https://developers.googleblog.com/en/a2a-a-new-era-of-agent-interoperability/)** (Google)
- **[AGNTCY](https://outshift.cisco.com/blog/building-the-internet-of-agents-introducing-the-agntcy)** (Cisco)

![MCP Architecture](https://www.descope.com/_next/image?url=https%3A%2F%2Fimages.ctfassets.net%2Fxqb1f63q68s1%2F2x3R1j8peZzdnweb5m1RK3%2Fa8628561358334a605e7f291560fc7cc%2FMCP_learning_center_image_1-min__1_.png&w=1080&q=75)

---

## 🎯 MCP on Databricks

**Databricks MCP Features:**
- ✅ **Pre-built MCP Server** - Automatically exposes UC Functions as MCP tools
- ✅ **Custom MCP Servers** - Build with Databricks Apps
- ⚠️ **Free Edition Limitation** - Databricks MCP Server not available

**BUT:** We can still learn MCP concepts and build a **custom MCP-style client** in Free Edition!

---

## 📚 What You'll Learn

In this notebook, we'll explore:

1. **MCP Protocol Basics** - Understand the standard
2. **Tool Schema Design** - MCP-compatible tool definitions
3. **Simple MCP Client** - Build a Python client (Free Edition compatible!)
4. **Tool Integration** - Connect to UC Functions using MCP patterns
5. **Real-World Example** - Maintenance agent with MCP-style tools

**Resources:**
- [MCP Official Docs](https://modelcontextprotocol.io/introduction)
- [MCP Spec](https://spec.modelcontextprotocol.io/)
- [Databricks MCP](https://docs.databricks.com/aws/en/generative-ai/mcp/)

In [ ]:
# Setup
%pip install --quiet requests pydantic typing-extensions
dbutils.library.restartPython()

In [ ]:
# Initialize
%run ../_resources/00-setup $reset_all_data=false

---

## 📚 Lesson 1: MCP Protocol Structure

MCP uses **JSON-RPC 2.0** for communication. Let's understand the message format:

### 🔄 Request Format

```json
{
  "jsonrpc": "2.0",
  "id": "request-123",
  "method": "tools/call",
  "params": {
    "name": "get_turbine_status",
    "arguments": {
      "turbine_id": "WT-042"
    }
  }
}
```

### ✅ Response Format

```json
{
  "jsonrpc": "2.0",
  "id": "request-123",
  "result": {
    "content": [
      {
        "type": "text",
        "text": "Turbine WT-042: Status OK, Power: 2.3 MW"
      }
    ]
  }
}
```

### 🛠️ Tool Definition Format

```json
{
  "name": "get_turbine_status",
  "description": "Get current status of a wind turbine",
  "inputSchema": {
    "type": "object",
    "properties": {
      "turbine_id": {
        "type": "string",
        "description": "Unique turbine identifier"
      }
    },
    "required": ["turbine_id"]
  }
}
```

In [ ]:
# Build a Simple MCP-Style Client

from typing import Dict, List, Any, Optional
from pydantic import BaseModel, Field
import json

class MCPToolSchema(BaseModel):
    """MCP-compatible tool schema"""
    name: str
    description: str
    inputSchema: Dict[str, Any]

class MCPTool:
    """Represents an MCP tool that wraps a UC Function"""
    
    def __init__(self, name: str, description: str, parameters: Dict[str, Any]):
        self.name = name
        self.description = description
        self.parameters = parameters
    
    def to_mcp_schema(self) -> Dict[str, Any]:
        """Convert to MCP tool schema format"""
        return {
            "name": self.name,
            "description": self.description,
            "inputSchema": {
                "type": "object",
                "properties": self.parameters,
                "required": list(self.parameters.keys())
            }
        }
    
    def __repr__(self):
        return f"MCPTool(name='{self.name}')"

class SimpleMCPClient:
    """Simple MCP-style client for UC Functions"""
    
    def __init__(self, catalog: str, schema: str):
        self.catalog = catalog
        self.schema = schema
        self.tools: List[MCPTool] = []
    
    def register_tool(self, tool: MCPTool):
        """Register a tool with the client"""
        self.tools.append(tool)
        print(f"✅ Registered tool: {tool.name}")
    
    def list_tools(self) -> List[Dict[str, Any]]:
        """List all tools in MCP format"""
        return [tool.to_mcp_schema() for tool in self.tools]
    
    def call_tool(self, tool_name: str, arguments: Dict[str, Any]) -> str:
        """Call a UC Function tool"""
        tool = next((t for t in self.tools if t.name == tool_name), None)
        if not tool:
            return json.dumps({"error": f"Tool '{tool_name}' not found"})
        
        try:
            # Build SQL query to call UC Function
            args_list = [f"'{v}'" if isinstance(v, str) else str(v) 
                        for v in arguments.values()]
            args_str = ", ".join(args_list)
            
            query = f"SELECT {self.catalog}.{self.schema}.{tool_name}({args_str}) as result"
            result = spark.sql(query).collect()[0]['result']
            
            # Return in MCP response format
            return json.dumps({
                "jsonrpc": "2.0",
                "result": {
                    "content": [
                        {
                            "type": "text",
                            "text": str(result)
                        }
                    ]
                }
            }, indent=2)
            
        except Exception as e:
            return json.dumps({
                "jsonrpc": "2.0",
                "error": {
                    "code": -32000,
                    "message": str(e)
                }
            }, indent=2)

print("✅ SimpleMCPClient class defined")

In [ ]:
# Example: Register UC Functions as MCP Tools

# Initialize MCP client
mcp_client = SimpleMCPClient(catalog=catalog, schema=db)

# Define tools in MCP format
turbine_status_tool = MCPTool(
    name="turbine_specifications_retriever",
    description="Retrieves current sensor readings and specifications for a wind turbine",
    parameters={
        "turbine_id": {
            "type": "string",
            "description": "Unique identifier for the turbine (format: UUID)"
        }
    }
)

maintenance_predictor_tool = MCPTool(
    name="turbine_maintenance_predictor",
    description="Predicts potential failures based on sensor readings",
    parameters={
        "avg_energy": {"type": "number", "description": "Average energy output"},
        "std_sensor_A": {"type": "number", "description": "Sensor A standard deviation"},
        "std_sensor_B": {"type": "number", "description": "Sensor B standard deviation"},
        "std_sensor_C": {"type": "number", "description": "Sensor C standard deviation"},
        "std_sensor_D": {"type": "number", "description": "Sensor D standard deviation"},
        "std_sensor_E": {"type": "number", "description": "Sensor E standard deviation"},
        "std_sensor_F": {"type": "number", "description": "Sensor F standard deviation"}
    }
)

# Register tools
mcp_client.register_tool(turbine_status_tool)
mcp_client.register_tool(maintenance_predictor_tool)

print("\n📋 Available MCP Tools:")
print(json.dumps(mcp_client.list_tools(), indent=2))

In [ ]:
# Test MCP Tool Calls

print("🧪 Testing MCP Tool Calls\n")

# Test 1: Get turbine status
print("=" * 60)
print("TEST 1: turbine_specifications_retriever")
print("=" * 60)

response1 = mcp_client.call_tool(
    "turbine_specifications_retriever",
    {"turbine_id": "004a641f-e9e5-9fff-d421-1bf88319420b"}
)
print(response1)

# Test 2: Predict maintenance
print("\n" + "=" * 60)
print("TEST 2: turbine_maintenance_predictor")
print("=" * 60)

response2 = mcp_client.call_tool(
    "turbine_maintenance_predictor",
    {
        "avg_energy": 0.9,
        "std_sensor_A": 2.2,
        "std_sensor_B": 2.6,
        "std_sensor_C": 2.1,
        "std_sensor_D": 2.2,
        "std_sensor_E": 2.6,
        "std_sensor_F": 2.1
    }
)
print(response2)

---

## 📚 Lesson 2: MCP Client Integration with LLMs

Now let's integrate our MCP client with an LLM to create a true MCP-style agent:

In [ ]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

class MCPAgent:
    """Agent that uses MCP-style tools"""
    
    def __init__(self, mcp_client: SimpleMCPClient, system_prompt: str):
        self.mcp_client = mcp_client
        self.system_prompt = system_prompt
        self.w = WorkspaceClient()
    
    def _format_tools_for_llm(self) -> str:
        """Format MCP tools for LLM understanding"""
        tools_desc = []
        for tool in self.mcp_client.tools:
            schema = tool.to_mcp_schema()
            params = ", ".join([
                f"{name}: {props.get('type', 'any')}"
                for name, props in schema['inputSchema']['properties'].items()
            ])
            tools_desc.append(
                f"- {tool.name}({params}): {tool.description}"
            )
        return "\n".join(tools_desc)
    
    def chat(self, user_message: str) -> str:
        """Chat with agent using MCP tools"""
        
        # Build prompt with tool descriptions
        full_prompt = f"""{self.system_prompt}

Available MCP Tools:
{self._format_tools_for_llm()}

When you need to use a tool, respond with:
TOOL_CALL: tool_name(arg1="value1", arg2="value2")

User: {user_message}
"""
        
        # Get LLM response
        response = self.w.serving_endpoints.query(
            name="databricks-meta-llama-3-1-70b-instruct",
            messages=[
                {"role": "system", "content": full_prompt}
            ],
            max_tokens=500
        )
        
        llm_response = response.choices[0].message.content
        
        # Check if tool call is needed
        if "TOOL_CALL:" in llm_response:
            # Parse and execute tool call
            # (Simplified - real implementation would use structured output)
            print(f"🔧 LLM requested tool call: {llm_response}")
            return f"Tool call detected. In production MCP, this would be executed automatically.\n\n{llm_response}"
        
        return llm_response

# Create MCP Agent
mcp_agent = MCPAgent(
    mcp_client=mcp_client,
    system_prompt="""You are a wind turbine maintenance assistant using MCP tools.
    Use the available tools to answer questions about turbines.
    Be concise and helpful."""
)

print("✅ MCP Agent initialized")

In [ ]:
# Test MCP Agent

test_queries = [
    "What is the current status of turbine 004a641f-e9e5-9fff-d421-1bf88319420b?",
    "Can you predict if any failures are likely?",
    "Explain what MCP protocol enables for agent systems"
]

print("🤖 Testing MCP Agent\n")

for i, query in enumerate(test_queries, 1):
    print("=" * 60)
    print(f"Query {i}: {query}")
    print("=" * 60)
    response = mcp_agent.chat(query)
    print(f"\n{response}\n")

---

## 🎯 Key Benefits of MCP

### 1️⃣ **Standardization**
- Same protocol across different AI applications
- Tools work with any MCP-compatible client
- Easier integration and maintenance

### 2️⃣ **Interoperability**
- Tools from different providers work together
- Mix and match tool sources
- Future-proof your agent systems

### 3️⃣ **Modularity**
- Add/remove tools without changing agent code
- Version tools independently
- Share tools across multiple agents

### 4️⃣ **Security & Governance**
- Standardized authentication
- Clear permission boundaries
- Audit trail for tool usage

---

## 🚀 Real-World MCP Use Cases

**✅ Already Working:**
- Claude Desktop with MCP servers
- Custom MCP servers (Python, TypeScript)
- Open-source MCP tool ecosystem

**🔜 Coming Soon:**
- More LLM providers adopting MCP
- Databricks native MCP in all editions
- Industry-standard MCP tool marketplace

---

## 💡 What We Built (Free Edition Compatible!)

Today you learned:

1. ✅ **MCP Protocol Basics** - JSON-RPC 2.0 format
2. ✅ **SimpleMCPClient** - Custom client for UC Functions
3. ✅ **Tool Registration** - MCP-style tool schemas
4. ✅ **MCPAgent** - LLM integration with MCP tools
5. ✅ **Testing** - Real tool calls via MCP pattern

**This works in Free Edition!** 🎉

The concepts and patterns you learned today will transfer directly to production MCP deployments when available.

---

## 📚 Next Steps

**Learn More:**
- [MCP Official Docs](https://modelcontextprotocol.io/)
- [MCP Python SDK](https://github.com/modelcontextprotocol/python-sdk)
- [Build MCP Servers](https://modelcontextprotocol.io/docs/develop/build-server)

**Practice:**
- Build more MCP tools
- Create custom tool schemas
- Experiment with different LLM integrations

**Production:**
- When available, migrate to Databricks MCP Server
- Deploy custom MCP servers
- Connect to ecosystem tools